## Configuração (Somente quando necessário)

In [ ]:
! pip install pymysql;

## Restaurar danco de dados

In [1]:
import pymysql
import os
import time
from datetime import datetime
import subprocess
from concurrent.futures import ThreadPoolExecutor

def listar_databases_atuais(cursor):
    cursor.execute('SHOW DATABASES')
    bancos_sommusgestor = [db[0] for db in cursor.fetchall() if 'sommusgestor' in db[0]]
    return bancos_sommusgestor

def deletar_database(cursor, nome):
    cursor.execute(f"DROP DATABASE {nome}")

def criar_database(cursor, nome):
    if(nome.startswith("sommusgestor") and not nome.endswith("_old")):
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {nome}")

def restaurar(banco, host, porta, usuario, password, diretorio):
    if(banco.startswith("sommusgestor") and not banco.endswith("_old")):
        comando = f'mysql -h {host} -P {porta} -u {usuario} -p{password} --default-character-set=utf8mb4 {banco} < {diretorio}/{banco}.sql'
        subprocess.run(comando, shell=True, encoding='utf-8')
        print(f"  🔵 - {banco}")


In [2]:
HOST = 'localhost'
PORTA = 3307
USUARIO = 'root'
PASSWORD = 'root'
PASTA = 'backups'
SUBPASTA = 'dev-andre_2024-08-21_19-07-03'
DIRETORIO_BACKUP = f'{PASTA}/{SUBPASTA}'

if __name__ == '__main__':
    try:
        with pymysql.connect(host=HOST, user=USUARIO, password=PASSWORD, port=PORTA).cursor() as cursor:
            mensagem_inicial = f"🟡 Restauração iniciada: {SUBPASTA}"
            print(mensagem_inicial)
            print("=" * (len(mensagem_inicial) + 2))
            bancos_sommusgestor = listar_databases_atuais(cursor)
            
            print("🔵 [1/3] - Deletando os bancos de dados")
            for banco in bancos_sommusgestor:
                deletar_database(cursor, banco)            

            arquivos_sql_backup = os.listdir(DIRETORIO_BACKUP)

            print("🔵 [2/3] - Criando os databases")
            for arquivo in arquivos_sql_backup:
                nome_arquivo_sem_extensao = os.path.splitext(arquivo)[0]
                criar_database(cursor, nome_arquivo_sem_extensao)        

            print("🔵 [3/3] - Restaurando os dados")
            for arquivo in arquivos_sql_backup:
                nome_arquivo_sem_extensao = os.path.splitext(arquivo)[0]
                restaurar(nome_arquivo_sem_extensao, HOST, PORTA, USUARIO, PASSWORD, DIRETORIO_BACKUP)

        mensagem_final = f"✅ Restauração finalizada em {datetime.now().strftime('%d/%m/%Y às %H:%M:%S')}"
        print("=" * (len(mensagem_final) + 2))
        print(mensagem_final)

    except Exception as e:
        print(f'⛔ [Exceção lançada] \n{e}')

 

🟡 Restauração iniciada: sommus-versionamento_2024-08-16_09-49-12
🔵 [1/3] - Deletando os bancos de dados
🔵 [2/3] - Criando os databases
🔵 [3/3] - Restaurando os dados
  🔵 - sommusgestor
  🔵 - sommusgestor_1
  🔵 - sommusgestor_hangfire
✅ Restauração finalizada em 20/08/2024 às 15:41:33
